# Motor de Búsqueda con BERT en Español
Este notebook implementa un motor de búsqueda utilizando el modelo BETO (BERT en Español).
Procesa un corpus de noticias en formato CSV y realiza búsquedas basadas en la similitud del texto.

In [1]:
# Instalar dependencias necesarias
%pip install transformers torch scikit-learn pandas


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Importar Librerías

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

c:\Users\guzma\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Cargar el Modelo BETO en Español

In [3]:
MODEL_NAME = "dccuchile/bert-base-spanish-wwm-cased"  # BETO

print("Cargando el modelo y tokenizer...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)
print("Modelo cargado exitosamente!")

Cargando el modelo y tokenizer...


c:\Users\guzma\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\guzma\.cache\huggingface\hub\models--dccuchile--bert-base-spanish-wwm-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertModel were not initialized from the model checkpoint at 

Modelo cargado exitosamente!


## 3. Cargar el Corpus CSV

In [4]:
# Cargar el archivo CSV
file_path = "../../../data/raw_data_corpus.csv"  # Reemplaza con la ruta de tu archivo CSV

df = pd.read_csv(file_path)
print("Primeras filas del corpus:")
df.head()

Primeras filas del corpus:


,Source,Title,Content,Section,URL,Date
0,Jornada,Récord de 96 mil asistentes para pelea Dubois ...,"Londres. Daniel Dubois, nueva estrella del box...",Sports,https://www.jornada.com.mx/2024/09/21/deportes...,21/09/2024
1,Jornada,"América no está para formar jugadores, dice ...",Siendo el América uno de los clubes que menos ...,Sports,https://www.jornada.com.mx/2024/09/21/deportes...,21/09/2024
2,Jornada,"Fátima Herrera, sin miedo a nada, hizo histor...",El surgimiento de referentes en el deporte de ...,Sports,https://www.jornada.com.mx/2024/09/21/deportes...,21/09/2024
3,Jornada,Pumas femenil deja escapar los tres puntos en CU,"Pese a generar diversas oportunidades de gol, ...",Sports,https://www.jornada.com.mx/2024/09/21/deportes...,21/09/2024
4,Jornada,Cae dupla Zverev-Alcaraz,La dupla Carlos Alcaraz-Alexander Zverev cayó ...,Sports,https://www.jornada.com.mx/2024/09/21/deportes...,21/09/2024


## 4. Generar los Embeddings BERT del Corpus

In [5]:
# Combinar columnas Title y Content para el procesamiento
corpus = df["Content"].fillna("") + " " + df["Title"].fillna("")

# Función para obtener embeddings BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()  # Promediar los embeddings

# Generar embeddings del corpus
print("Generando embeddings para el corpus...")
corpus_embeddings = [get_bert_embedding(text) for text in corpus]
print("Embeddings generados!")

Generando embeddings para el corpus...
Embeddings generados!


## 5. Función de Búsqueda de Textos Similares

In [10]:
def get_search_results(input_text, top_n=3):
    if input_text.strip() == "":
        return []
    
    # Obtener embedding para la consulta
    query_embedding = get_bert_embedding(input_text)
    
    # Calcular similitud coseno
    similarities = [cosine_similarity(query_embedding, doc_emb)[0][0] for doc_emb in corpus_embeddings]
    
    # Ordenar resultados por similitud
    sorted_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)
    
    # Retornar los títulos y puntuaciones
    results = []
    for i in sorted_indices[:top_n]:
        results.append([df.loc[i, "Title"], similarities[i]])
    
    return results

## 6. Probar la Función de Búsqueda

In [13]:
# Prueba de búsqueda
query = "Esta es la audiencia que quiere conquistar el iPhone 16 Apple es consciente de que muchos usuarios de iPhone 15 difícilmente van a renovar su smartphone"
print(f"Resultados para la consulta: '{query}'\n")

results = get_search_results(query)
for title, score in results:
    print(f"- {title} (Similitud: {score:.4f})")

Resultados para la consulta: 'Esta es la audiencia que quiere conquistar el iPhone 16 Apple es consciente de que muchos usuarios de iPhone 15 difícilmente van a renovar su smartphone'

- Esta es la audiencia que quiere conquistar el iPhone 16 (Similitud: 0.9826)
- ¿Cuánto me da Apple por mi iPhone para abonar en la compra de uno nuevo? (Similitud: 0.9354)
- iOS 18, qué incluye esta nueva versión y cómo actualizar tu iPhone (Similitud: 0.9329)


## 7. Evaluación del Modelo con F1-Score (Opcional)

In [12]:
def evaluate_f1_score(true_titles, predicted_titles):
    y_true = [1 if title in true_titles else 0 for title in predicted_titles]
    y_pred = [1] * len(predicted_titles)
    return f1_score(y_true, y_pred, average="macro")

# Ejemplo de uso (ground truth necesario)
true_titles = ["Récord de 96 mil asistentes para pelea Dubois contra Joshua hoy en Wembley"]
predicted_titles = [result[0] for result in results]
f1 = evaluate_f1_score(true_titles, predicted_titles)
print(f"\nF1-Score: {f1:.4f}")


F1-Score: 0.2500
